# Filtering NHC Hurricane Archive Based on HRRR Domain
Data Availability: [NHC Hurricane Archive](https://www.nhc.noaa.gov/data/hurdat/)
- See format details in `format.pdf`

There are two types of rows for this dataset
- Type 1 Format: entries, date, time,
- Type 2 Format: record_identifier, status, lat, lon, vmax, pres, 34ne, 34se, 34sw, 34nw, 50ne, 50se, 50sw, 50nw, 64ne, 64se, 64sw, 64nw, rmax
   - Note: Date is YYYMMDD and Time is UTC

A note on missing values
- Radii at 34kt, 50kt, and 64kt were not recorded until hurricane AL012004 - ALEX in 07/31/2004 (can be used with HRRR, since HRRR starts in 2014)
- Radii at maximum windspeed (Rmax) was not recorded in many hurricanes until AL012021 - ANA in 05/20/2021 (requires replacement with NaN)

In [75]:
from herbie import Herbie
import numpy as np
import pandas as pd

In [76]:
filename = 'hurdat2-1851-2023-051124.txt'
missingVal = -999
basin='AL'
minDate = 20140730
columnHeaders = ['id', 'name', 'entries', 'date', 'time', 'record_identifier', 'status', 'lat', 'lon', 'vmax', 'pres', '34ne', '34se', '34sw', '34nw', '50ne', '50se', '50sw', '50nw', '64ne', '64se', '64sw', '64nw', 'rmax']

## Filtering NHC Hurricane Archive Data

In [77]:
# Reformatting input file to csv and creating a new file named 'data.csv'
with open(filename, 'r') as f_in, open('data.csv', 'w') as f_out:
    tmp = ""
    for line in f_in:
        line = line.replace(' ', '')  # Remove spaces
        if line.startswith(basin):
            tmp = line.strip()  # Store the basin line
        else:
            if int(line[:8]) >= minDate: # Filter by minimum HRRR data
                line = tmp + line  # Prepend the last stored basin
                f_out.write(line)  # Write filtered line 

In [78]:
data = pd.read_csv('data.csv', header=None, names=columnHeaders)
data.replace(-999, np.nan, inplace=True) 
data

,id,name,entries,date,time,record_identifier,status,lat,lon,vmax,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,rmax
0,AL032014,BERTHA,47,20140730,0,NaN,LO,9.6N,41.5W,30,...,0,0,0,0,0,0,0,0,0,NaN
1,AL032014,BERTHA,47,20140730,600,NaN,LO,9.7N,43.0W,30,...,0,0,0,0,0,0,0,0,0,NaN
2,AL032014,BERTHA,47,20140730,1200,NaN,LO,9.8N,44.7W,30,...,0,0,0,0,0,0,0,0,0,NaN
3,AL032014,BERTHA,47,20140730,1800,NaN,LO,10.0N,46.4W,30,...,0,0,0,0,0,0,0,0,0,NaN
4,AL032014,BERTHA,47,20140731,0,NaN,LO,10.4N,48.0W,35,...,40,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,AL212023,TWENTY-ONE,6,20231023,1800,NaN,TD,11.5N,83.2W,25,...,0,0,0,0,0,0,0,0,0,60.0
5594,AL212023,TWENTY-ONE,6,20231024,0,NaN,TD,12.2N,83.4W,25,...,0,0,0,0,0,0,0,0,0,60.0
5595,AL212023,TWENTY-ONE,6,20231024,130,L,TD,12.4N,83.5W,25,...,0,0,0,0,0,0,0,0,0,60.0
5596,AL212023,TWENTY-ONE,6,20231024,600,NaN,TD,13.0N,83.8W,25,...,0,0,0,0,0,0,0,0,0,60.0


In [79]:
# Convert latitude
data['lat'] = data['lat'].str[:-1].astype(float)


# Convert longitude
data['lon'] = '-' + data['lon'] 
data['lon'] = data['lon'].str[:-1].astype(float)

First, we will load a HRRR data sample using Herbie to pick points for our NCH tracks

- Note: See `herbieLoading.ipynb` for a guide on how HRRR data can be loaded using Herbie

In [80]:
H = Herbie('2021-07-19', model='hrrr', product='sfc', fxx=0)
ds = H.xarray("TMP:2 m above")

points = data[['lat', 'lon']]
points = points.rename(columns={'lat': 'latitude', 'lon': 'longitude'})

nearest = ds.herbie.pick_points(points, max_distance=6000)
nearest


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Jul-19 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


<xarray.Dataset>
Dimensions:              (point: 5598)
Coordinates:
    time                 datetime64[ns] 2021-07-19
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 2.0
    latitude             (point) float64 21.14 21.14 21.14 ... 23.1 23.17 23.28
    longitude            (point) float64 287.7 287.7 287.7 ... 278.4 278.0 277.2
    valid_time           datetime64[ns] 2021-07-19
    gribfile_projection  object None
    point_grid_distance  (point) float64 3.533e+03 3.379e+03 ... 1.101e+03
    point_latitude       (point) float64 9.6 9.7 9.8 10.0 ... 12.4 13.0 13.5
    point_longitude      (point) float64 -41.5 -43.0 -44.7 ... -83.5 -83.8 -84.4
Dimensions without coordinates: point
Data variables:
    t2m                  (point) float32 301.6 301.6 301.6 ... 301.4 301.1 302.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              C:\Users\joebe\data\hrrr\20210719\subset_6eeffd5...
    search:                  TMP:2 m above

### We filter based on a desired 'distance from nearest gridpoint' threshold

In [81]:
# Change threshold for points further away from the border of HRRR domain
threshold = 3
indeces = [] # Stores deleted indeces

# Loop over each point and collect indeces for deletion
for i in range(data.shape[0]):
    lat = nearest.point_latitude[i].item()
    lon = nearest.point_longitude[i].item()
    distance = nearest.point_grid_distance[i].item()
    if distance > threshold:
        indeces.append(i)

# Delete points outside of the domain
data.drop(indeces, inplace=True)

In [85]:
# Count the number of hurricane tracks
print(f'Number of hurricane tracks available: {data['id'].nunique()}')
data

Number of hurricane tracks available: 79


,id,name,entries,date,time,record_identifier,status,lat,lon,vmax,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,rmax
20,AL032014,BERTHA,47,20140803,1800,NaN,TS,22.7,-72.5,45,...,50,0,0,0,0,0,0,0,0,NaN
21,AL032014,BERTHA,47,20140804,0,NaN,TS,24.1,-73.1,55,...,40,40,40,0,0,0,0,0,0,NaN
22,AL032014,BERTHA,47,20140804,600,NaN,TS,25.4,-73.5,60,...,40,40,40,0,0,0,0,0,0,NaN
23,AL032014,BERTHA,47,20140804,1200,NaN,HU,26.8,-73.6,70,...,40,40,40,0,0,20,20,0,0,NaN
24,AL032014,BERTHA,47,20140804,1800,NaN,HU,28.5,-73.6,70,...,40,40,40,0,0,20,20,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5438,AL162023,OPHELIA,15,20230923,1800,NaN,TS,36.3,-77.5,40,...,40,0,0,0,0,0,0,0,0,100.0
5439,AL162023,OPHELIA,15,20230924,0,NaN,EX,36.8,-77.6,30,...,0,0,0,0,0,0,0,0,0,150.0
5440,AL162023,OPHELIA,15,20230924,600,NaN,EX,37.2,-77.6,25,...,0,0,0,0,0,0,0,0,0,150.0
5441,AL162023,OPHELIA,15,20230924,1200,NaN,EX,37.9,-77.4,20,...,0,0,0,0,0,0,0,0,0,200.0


### Checking percentage of missing values for relevant columns

In [98]:
empty_vmax = (data['vmax'] == -999).sum() / data.shape[0]
empty_pres = (data['pres'] == -999).sum() / data.shape[0]

empty_rmax = (data['rmax'].isnull()).sum() / data.shape[0]
empty_34 = (data['34nw'] == -999).sum() / data.shape[0]
empty_50 = (data['50nw'] == -999).sum() / data.shape[0]
empty_64 = (data['64nw'] == -999).sum() / data.shape[0]


print(f"Ratio of missing values:\nvmax: {empty_vmax}\npres: {empty_pres}\nrmax: {empty_rmax}\n34: {empty_34}\n50: {empty_50}\n64: {empty_64}")

Ratio of missing values:
vmax: 0.0
pres: 0.0
rmax: 0.7433358720487433
34: 0.0
50: 0.0
64: 0.0


### Save dataset to csv

In [ ]:
data.to_csv('filtered_data.csv', index=False)